### Implementing the EGM in a Model of Human Capital

 <font size="2"> John Green <br> May 2023 <font>


In this notebook I will demonstrate the solution method for my second year paper, "Skills-Biased Technological Change and the Human Capital Production Function." In the paper I estimate a model of life-cycle labor supply and consumption with human capital similar to that of Imai and Keane (2004). I estimate this on data from 1979 and 1997, and compare the parameters for the human capital production function (HCPF) in order to test the skills-biased technological change (SBTC) hypothesis. I solve the model using the Endogenous Grid Method of Carroll (2006), which provides a significant speedup (by about a factor of 50) over using backwards-recursion of the value function. The gain in computation time is in addition to a gain in accuracy, since the value function iteration seems prone to getting stuck at local minima. However, because of the ``twisted grid" problem which occurs in the endogenous grid step, implementation is not straightforward.

I will first present the model, then show my first order conditions and give some intuition for the solution, and then demonstrate how to use the EGM to solve the agent's problem. Full details as well as the results of the estimation are provided in the full paper, which is available upon request.

#### The model
In each annual period agents choose their consumption and their labor supply. Agents derive current-period utility from consumption and disutility from labor, following the classic additive CRRA form, as used in MaCurdy (1981) and Altonji (1986). Utility from consumption is:

\begin{equation}
\tag{1}
u(c) = \dfrac{c^{1-\gamma}}{1 - \gamma}
\end{equation}

Disutility from labor is:
\begin{equation}
\tag{2}
v(h) = b \dfrac{h^{1+ \eta}}{1 + \eta}
\end{equation}

Labor income is determined by the number of hours worked as well as the level of human capital, which determines workers' wages:
\begin{equation}
\tag{3}
Y_t = K_t h_t
\end{equation}

Assets are determined by the intertemporal budget constraint:
\begin{equation}
\tag{4}
A_{t+1} = \left( A_t + Y_t - c_t \right) R
\end{equation}
Where $R$ is the risk-free interest rate and is set to 1.05.

Human capital is accumulated by learning-by-doing. That is, agents increase their next period human capital by working in the current period. At the same time, the existing human capital stock depreciates. There is also a stochastic shock. The deterministic portion of human capital is:
\begin{equation}
\tag{5}
K_{\bar{t}} = \delta K_t + G(h_t,s) K_t
\end{equation}

where $G(\cdot,\cdot)$ is a function increasing in hours worked and decreasing in age ($h$ and $s$ respectively). The notation $K_{\bar{t}}$ indicates the level of human capital at the *end* of period $t$, after the control variables have been realized but before the stochastic shock. Then human capital at the beginning of the next period is:
\begin{equation}
\tag{6}
K_{\underline{t+1}} = \epsilon_{t+1}  K_{\bar{t}}
\end{equation}

This distinction between end-of-period deterministic human capital and the beginning-period human capital will be useful when we implement the EGM. 

The HCPF $G(\cdot,\cdot)$ is increasing in hours worked and decreasing in age. It is specified as:
\begin{equation}
\tag{7}
G(h,t) = \omega_0(1 - \omega_1(s-19)) \left[ (h+d)^{\alpha} - \omega_2(h+d)\right]
\end{equation}

This form is similar to that of Imai and Keane (2004), and is meant to capture several features of the data:
* Wages grow at a slower rate as workers get older, captured by  $-\omega_1(s-19)$
* Returns to hours worked are not linear, which is why we include $\alpha<1$
* At high levels of hours, returns may be negative, which is reflected by $-\omega_2(h+d)$
* $d$ captures a lower-bound to wages

The lower bound is likely an artifact of the minimum wage, and in future work I would like to model this explicitly.

Putting all of this together, agents' problem can be written in the Bellman equation form:
\begin{equation}
\tag{8}
    \begin{split}
       & V_t(A_t,K_t) =  \text{max}_{c_t,h_t} \left\{ u(c_t) - v(h_t) + \beta \mathbb{E}_t V_{t+1} \left(A_{t+1}, K_{t+1} \right) \right\} \\
                    & =  \text{max}_{c_t,h_t} \left\{ u(c_t) - v(h_t) + 
                    \beta \mathbb{E}_t V_{t+1} \left( R(A_t + K_t h_t - c_t), \epsilon_{t+1}( \delta K_{\bar{t}} + G(h_t,s) K_{\bar{t}}) \right) \right\}
    \end{split}
\end{equation}

Where $\beta$ gives the agent's discount factor.

#### First order conditions

The solution method will take advantage of the problem's first order conditions. The FOC with respect to consumption is:
\begin{equation}
\tag{9}
u'(c_t) = \beta R \mathbb{E} \left[ u'(c_{t+1}) \right]
\end{equation}
which is the classic consumption Euler equation.

The FOC with respect to labor supply is slightly more complicated:
\begin{equation}
\tag{10}
v'(h_t) = K_t u'(c_t) + \beta \kappa(h_t, K_t, s) \mathbb{E} \left[ \epsilon_t \dfrac{\partial V_{t+1}}{\partial K_{t+1}} \right]
\end{equation}

$\frac{\partial V_{t}}{\partial K_{t}}$ takes the form:
\begin{equation}
\tag{11}
\dfrac{\partial V_{t}}{\partial K_{t}} = h_t u'(c_{t}) + \beta \lambda (h_t,s) \mathbb{E} \left[ \epsilon_{t+1}  \dfrac{\partial V_{t+1}}{\partial K_{t+1}} \right]
\end{equation}

Note that in our terminal period $T$, the RHS term $\mathbb{E} \left[ \epsilon_{t+1}  \frac{\partial V_{t+1}}{\partial K_{t+1}} \right] = 0$, which will be useful.

Finally, we will need to invert the law of motion for human capital. Define:
\begin{equation}
    \tag{12}
    \begin{split}
        K_{\underline{t}} & = K_{\bar{t}}( \delta + \omega_0(1 - \omega_1(s-19))[(h_t + d)^{\alpha} - \omega_2(h_t+d)])^{-1} \\
        & \equiv \Psi(K_{\bar{t}}, h_t,s)
    \end{split}
\end{equation}

So that eqn. (10) can be rewritten as:
\begin{equation}
\tag{13}
v'(h_t) = \Psi(K_{\bar{t}}, h_t,s) u'(c_t) + \beta \kappa(h_t, \Psi(K_{\bar{t}}, h_t, s), s) \mathbb{E} \left[ \epsilon_t \dfrac{\partial V_{t+1}}{\partial K_{t+1}} \right]
\end{equation}

#### Implementing the EGM

I will now solve the agent's problem using the EGM, in order to get an approximation of the policy function for consumption and labor supply. First, some basic setup.

In [1]:
# Import necessary packages here
import math as mat
import numpy as np
import scipy
import pandas as pd
import xarray as xr
from scipy.optimize import root_scalar
from scipy import interpolate
from numpy import vectorize
from scipy.optimize import minimize_scalar
from scipy.interpolate import griddata
from scipy.interpolate import LinearNDInterpolator
import statsmodels.api as sm
from scipy.interpolate import interp2d
from scipy.interpolate import RegularGridInterpolator

# Now declare the global variables
# risk-free rate
r = 0.05
R = 1 + r
# Number of grid points for 
nA = 20 # assets
nK = 20 # human capital
nq = 6 # quadrature integration
# minimum consumption
cmin = 1e-5
# and maximum work
hmax = 5096
# Agents live from age 20 to 65
T = 65
t0 = 20
lifespan = T- t0

# Preset variables to be estimated
# note: in the paper I estimate for 4 separate education groups. Here I will use the values for college graduates.

# utility function parameters
gamma = 0.68 # curvature of utility from consumption
eta = 0.87 # curvature of disutility from labor
b = 5.1e-5 # scale parameter on labor disutility
beta = .986
# HCPF parameters
delta = .303
omega0 = .13
omega1 = 1e-3
alpha = .24
omega2 = 4.1e-4
d=512
# sd of wage shocks
sigma = 8.06e-2

Now I will define some of the functions from above, both from the structural model and those that are used in calculating the FOCs and in other intermediate steps.

In [2]:
# Utility functions:
def utility(c,gamma):
    if c < cmin:
        # illegal value of c; give a large negative utility
        print("invalid value of c")
        return -10000
    elif c >= cmin and gamma != 1:
        return (c**(1 - gamma))/(1-gamma)
    else:
        # case of log utility
        return mat.log(c)
    
def getMargUtilityC(c,gamma):
    if c < cmin:
        # illegal value of c; give an error
        # print("invalid value of c")
        return 999
    elif c >= cmin and gamma != 1:
        # print("normal value of c")
        return c**(-gamma)
    else:
        # case of log utility
        # print("log utility")
        return 1/c

def disutility(h,b,eta):
    return b*(h**(1+eta))/(1+eta)

def getMargDisutilityH(h,b,eta):
    return b*(h**eta)

def gFunc(h,K,age,delta,omega0,omega1,alpha,omega2,d):
    G =  omega0*(1+omega1*(age-19))*((h+d)**alpha - omega2*(h+d))
    return delta*K + G*K

# Now define functions relating to the FOCs:
# Tau
def taufunc(age,omega0,omega1):
    return omega0*(1+omega1*(age-19))

# Psi (inverse law of motion for human capital)
def psifunc(Kend, h, this_tau, d, alpha, omega2, delta):
    this_chi = ((h+d)**alpha - omega2*(h+d))
    return Kend/(delta + this_tau*this_chi)

# lambda
def lambdafunc(h, this_tau, omega2, d, alpha, delta):
    return delta + this_tau*((h + d)**alpha - omega2*(h+d))

# kappa
def kappafunc(Kstart, h, this_tau, d, omega2, alpha):
    return this_tau*(alpha*((h+d))**(alpha-1)-omega2)*Kstart

Our final piece of setup is to establish the exogenous grid of state variables, assets and human capital, over which we would like to get our policy function approximation. First, I define a function to calculate the grids, given minimum and maximum values, number of points, and a method for the steps.

In [3]:
# Write a function to get the grids
def GetGrid(min, max, points, method):

  # MMake sure min and max are arrays of the same length
  if len(min) != len(max):
    print("Error: min and max must be arrays of the same length.")
    exit()

  # get span in each period
  span = max - min

  # Get the grid using log steps
  if method == "log":
    loggrid = np.full((points, len(span)), np.nan)
    for i in range(len(min)):
      vec = np.linspace(np.log(1), np.log(1 + span[i]), points)
      loggrid[:, i] = vec

    # then exponentiate
    grid = np.exp(loggrid) - 1

    # add back in the min
    min_mat = np.full((points, len(span)), min)
    grid = grid + min_mat

    # grid[1,] == asset_lb
    # round(grid[points,], 2) == round(asset_ub, 2)
    # all good
  elif method == "linear":  # if we want to do linear interpolation
    grid = np.full((points, len(span)), np.nan)
    for i in range(len(min)):
      vec = np.linspace(min[i], max[i], points)
      grid[:, i] = vec
  else:
    print("error: method invalid. Choose 'log' or 'linear'")

  return grid

I set \$4 as the minimum value of the human capital grid. This is slightly below the minimum wage in the time period we are studying, allowing for some below-minimum earnings (due, for example, to black market work). I set the minimum as \$30 in the first period, which increases to \$100 by period $T$. This is arbitrary, but should capture well the range of earnings we see in the data.

In [4]:
kmin = np.repeat([4],lifespan)
kmax = np.linspace(30, 100, 45)
k_grid = GetGrid(kmin,kmax,nK,"linear")
# k_grid

I impose some mild liquidity constraints. Individuals are allowed to have negative assets up to twice their full-time earnings in all periods except for $T-1$, when they are allowed to have one year of full time earnings. Thus the lower bound for assets is defined by the borrowing constraint for those with the highest level of human capital on our grid. The upper bound of assets is set to \$250 thousand; this is arbitrary, but captures most of the observations in the data.

In [5]:
# minimum level of assets on our grid 
amin = np.full(lifespan, np.nan)
amin[len(amin)-1] = -k_grid[nK-1,len(amin)-1]*2040
for i in (range(len(amin)-1)):
    amin[i] = -k_grid[nK-1,i]*2040*2
amax = np.repeat([250000],lifespan)
a_grid = np.ascontiguousarray(GetGrid(amin, amax, nA, "linear"))
# a_grid[0,:]

There are two final pieces of setup. First, we need to discretize the distribution for our wage shocks, and then we need establish the arrays and vectors we will use in our solution.

In [6]:
def discretize_log_distribution(points, mean, sd):
  var = sd ** 2
  Z = np.full(points + 1, np.nan)
  EVbetweenZ = np.full(points, np.nan)

  # manually set upper bounds at +/- 3
  Z[0] = -3
  Z[points] = 3

  # Get the cutoff points
  prob = 1 / points
  for i in range(points - 1, 0, -1):
    Z[i] = scipy.stats.norm.ppf(prob * i)

  # get mean value in each interval (see Adda & Cooper page 58)
  for i in range(points):
    ev = (scipy.stats.norm.pdf(Z[i]) - scipy.stats.norm.pdf(Z[i + 1])) / (scipy.stats.norm.cdf(Z[i + 1]) - scipy.stats.norm.cdf(Z[i]))
    pt = sd * ev + mean
    EVbetweenZ[i] = pt

  quad2 = np.exp(EVbetweenZ)
  return quad2

quad = discretize_log_distribution(nq, -(1/2)*sigma**2, sigma)

# Now set up empty matrices and vectors that we will use
 # set up empty matrices 
policyC   = np.full((nA,nK,lifespan), np.nan)
policyH   = np.full((nA,nK,lifespan), np.nan)
dVdK      = np.full((nA,nK,lifespan), np.nan)
E_dVdK    = np.full((nA,nK,lifespan), np.nan)
E_eps_dVdK= np.full((nA,nK,lifespan), np.nan)
uPrime    = np.full((nA,nK,lifespan), np.nan)
E_uPrime  = np.full((nA,nK,lifespan), np.nan)
lc        = np.full((nA,nK,lifespan), np.nan)
extrap    = np.full((nA,nK,lifespan), np.nan)
replace   = np.full((nA,nK,lifespan-1), np.nan)
  
# vectors to store input for each E_{} value
Uprime_nextvec    = np.full(nq, np.nan) 
dVdK_nextvec      = np.full(nq, np.nan) 
eps_dVdK_nextvec  = np.full(nq, np.nan) 
E_dVdK_nextvec    = np.full(nq, np.nan) 
  
# In periods T-1 and prior, we will not have a grid, but a data frame of 
# nA*nK points
# rows for R^{-1}A_endofperiod, K_endofperiod, policyC, policyH,
# uPrime, dVdK, E_dVdK, E_eps_dVdK, E_uPrime
policy_df = np.full((nA*nK,13,lifespan-1), np.nan)

# Get minimum utility and marginal value at the constraint
umin = utility(cmin,gamma) - disutility(hmax,b,eta);
uprimemin = getMargUtilityC(cmin,gamma); 

We now have our grids and all of our variables set, and are ready to solve our problem. First we will solve the problem in period $T$. 

#### Period $T$

The agent's problem ends in period $T$, and for now we use no terminal value function. Thus the agent will consume all of their total resources (assets and labor income), and in practice only needs to choose the optimal labor supply (in the future I plan to implement a terminal period value function, representing retirement, bequest motive, etc.).

We will first set up a function from eqn. (10) which we can use to solve for $h_T^*$. Recalling that the second term on the RHS of (10) is equal to 0 since there is no value from human capital past period $T$, this is:

\begin{equation}
\tag{14}
v'(h_T) - K_T u'(A_T + h_T K_T) = h_T^{\eta} - K_T (A_T + h_T K_T)^{-\gamma} = 0
\end{equation}

Unfortunately, eqn. (14) has no analytical solution, so we will use a root-finder. We loop through every point on our grid, plug $A$ and $K$ into eqn. (14), and then get the optimal labor supply. We will first check whether the agent is liquidity constrained.

In [7]:
# Define function for eqn. (14)
def hFuncTerminal(h,A,K,b,gamma,eta):
    c = A + h*K
    dis = getMargDisutilityH(h,b,eta)
    util = getMargUtilityC(c,gamma)
    return dis - util*K

for ixa in range(nA): # looping through asset points
    # First, find the optimal labor supply for each K grid point at this A
    for ixk in range(nK): # looping through human capital points
        ub = hmax*k_grid[ixk,lifespan-1]+a_grid[ixa,lifespan-1]
        # print(ub)
        if ub < cmin:
            # liquidity constrained
            policyC[ixa,ixk,lifespan-1] = cmin
            policyH[ixa,ixk,lifespan-1] = hmax
            lc[ixa,ixk,lifespan-1] = 1
        
            # fill in uPrime and dVdK
            uPrime[ixa,ixk,lifespan-1] = uprimemin
            dVdK[ixa,ixk,lifespan-1]   = hmax*uprimemin
        else:
            # not liquidity constrained
            # get minimum hours
            hmin = (2*cmin-a_grid[ixa,lifespan-1])/k_grid[ixk,lifespan-1]+.1
            hmin = hmin if hmin > 0 else 0.1
            
            p1 = hFuncTerminal(hmin,a_grid[ixa,lifespan-1],k_grid[ixk,lifespan-1],b,gamma,eta)
            p2 = hFuncTerminal(hmax,a_grid[ixa,lifespan-1],k_grid[ixk,lifespan-1],b,gamma,eta)

            if p1*p2>0: # either both positive or both negative
              h = hmin if abs(p1)<abs(p2) else hmax 
            elif p1*p2 < 0: # endpoints are of different signs
              out = root_scalar(hFuncTerminal, bracket=[hmin, hmax], args=(a_grid[ixa, lifespan-1], 
                                k_grid[ixk, lifespan-1], b, gamma, eta), method='brentq', options={'maxiter': 500, 'xtol': 1e-12})         
              h = out.root
            else: # case where p1*p2=0 (almost surely won't happen)
              h = hmin if p1==0 else hmax
            # now fill in the policy functions
            policyH[ixa,ixk,lifespan-1] = h
            policyC[ixa,ixk,lifespan-1] = h*k_grid[ixk,lifespan-1]+a_grid[ixa,lifespan-1]
            lc[ixa,ixk,lifespan-1] = 0
            # now fill in Uprime and dVdK
            uPrime[ixa,ixk,lifespan-1] = getMargUtilityC(policyC[ixa,ixk,lifespan-1],gamma)
            dVdK[ixa,ixk,lifespan-1]   = policyH[ixa,ixk,lifespan-1]*uPrime[ixa,ixk,lifespan-1]
    # Once we have looped through human capital once and filled in the grid for a given level of assets, 
    # we can loop through again and fill in our expectations grid, holding this A fixed.

    for ixk in range(nK):
         nextK = quad*k_grid[ixk,lifespan-1]
         # If HC is off the grid, force it to stay on.
         nextK = np.where(nextK <= k_grid[0,lifespan-1], k_grid[0,lifespan-1]+0.01, nextK)
         nextK = np.where(nextK >= k_grid[nK-1,lifespan-1], k_grid[nK-1,lifespan-1]-0.01, nextK)

         f = interpolate.interp1d(k_grid[:,lifespan-1], uPrime[ixa,:,lifespan-1])
         Uprime_nextvec = f(nextK)
         f = interpolate.interp1d(k_grid[:,lifespan-1], dVdK[ixa,:,lifespan-1])
         dVdK_nextvec  = f(nextK)
         for i in range(len(dVdK_nextvec)):
              eps_dVdK_nextvec[i] = quad[i] * dVdK_nextvec[i]
        
         E_uPrime[ixa,ixk,lifespan-1] = np.mean(Uprime_nextvec)
         E_dVdK[ixa,ixk,lifespan-1] = np.mean(dVdK_nextvec)
         E_eps_dVdK[ixa,ixk,lifespan-1] = np.mean(eps_dVdK_nextvec)

Write out this last step in markdown.

We now have the problem in period $T$ solved, and can move on to period $T-1$.

#### Period $T-1$

As discussed, our use of the EGM will result in the ``twisted grid" problem. In order to fill in the exogenous grid from our endogenous grid, we will use a triangulation based on our endogenous values. One issue that we will confront is the necessity of filling in values of our exogenous grid not covered by our endogenous values. This is required, because if we simply drop grid points outside of the endogenous values, our grid will continuously narrow. In order to do this, I use a simple polynomial approximation, where the policy function is a function of $A$, $K$, $A\times K$, $A^2$, and $K^2$. This is not ideal, as there are well known shortcomings to using a polynomial approximation. The approximation may be very accurate on the domain of the endogenous values ($R^2 > 0.99$), outside of the domain it may not be accurate. A better solution is needed in the future.

First, I set up a data frame for the polynomial function.

In [8]:
# set up the polynomial df
polynomial_df = np.empty((nA*nK, 5, lifespan-1))
for ixt in range(0, lifespan-1):
    polynomial_df[:, 0, ixt] = np.tile(a_grid[:, ixt], len(k_grid[:, ixt]))
    polynomial_df[:, 1, ixt] = np.kron(k_grid[:, ixt], np.repeat(1, len(a_grid[:, ixt]))) # x2
    polynomial_df[:, 2, ixt] = polynomial_df[:, 0, ixt] * polynomial_df[:, 1, ixt] # x3
    polynomial_df[:, 3, ixt] = (polynomial_df[:, 0, ixt]**2) / 1e5 # x4
    polynomial_df[:, 4, ixt] = polynomial_df[:, 1, ixt]**2 # x5

Now we enter period $T-1$ , first melting our values from the beginning of period $T$ to be our values at the end of period $T-1$

In [9]:
# set our time indicator
ixt = lifespan-1-1

# end of period assets
policy_df[:,0,ixt] = np.tile(a_grid[:,ixt+1]*(R**(-1)),len(k_grid[:,ixt+1])) # unwind the interest payment to get assets at end of T-1

# end of period human capital
policy_df[:,1,ixt] = np.kron(k_grid[:,ixt+1], np.repeat(1,len(a_grid[:,ixt+1])))
# here we are treating realized HC in T as the deterministic HC at the end of T-1

# now melt the expected values we calculated
df = pd.DataFrame(E_uPrime[:, :, ixt+1])
policy_df[:,2, ixt] = pd.melt(df)["value"]
df = pd.DataFrame(E_dVdK[:, :, ixt+1])
policy_df[:,3, ixt] = pd.melt(df)["value"]
df = pd.DataFrame(E_eps_dVdK[:, :, ixt+1])
policy_df[:,4, ixt] = pd.melt(df)["value"]

# a few other values
df = pd.DataFrame(extrap[:, :, ixt+1])
policy_df[:,5, ixt] = pd.melt(df)["value"]
df = pd.DataFrame(lc[:, :, ixt+1])
policy_df[:,6, ixt] = pd.melt(df)["value"]

We now have a series of points $\left\{A_{\overline{T-1}},K_{\overline{T-1}} \right\}$, with the associated values of $\mathbb{E} \left( u'(c_T) \right)$, $\mathbb{E} \left( \frac{\partial V}{\partial K} \right)$, and $\mathbb{E} \left( \epsilon \frac{\partial V}{\partial K} \right)$. Essentially we will now work backwards to determine what the optimal controls $c_{T-1}^*$ and $h_{T-1}^*$, as well as the beginning of period $T-1$ state variables $\left\{A_{\underline{T-1}},K_{\underline{T-1}} \right\}$ *must have been* in order to arrive at the exogenous grid points we set for period $T$.

Note that we can solve eqn. (9) for $c_t$:

\begin{equation}
\tag{15}
c_t = (\beta R \mathbb{E} u'(c_{t+1}))^{-1/\gamma}
\end{equation}

Since we have $ \mathbb{E} u(c_{t+1})$ for each endogenous point, we can calculate $c_{T-1}^*$ very quickly for each of our points, and then fill in the marginal utility from that consumption (which we will use when we solve period $T-2$).

In [10]:
# Now get that period's optimal consumption using the inverted FOC
policy_df[:, 7, ixt] = (beta*R*policy_df[:,2,ixt])**(1/(-gamma));
# policy_df[:, 7, lifespan-1-1]
# and get marginal utility:
for i in range(policy_df.shape[0]):
    # print(i)
    policy_df[i,8, ixt] = getMargUtilityC(policy_df[i, 7, ixt], gamma)

Now that we have the optimal level of consumption, we can use our first order condition to solve for labor supply. Recall eqn. (13):

\begin{equation}
\tag{13}
v'(h_t) = \Psi(K_{\bar{t}}, h_t,t) u'(c_t) + \beta \kappa(h_t, \Psi(K_{\bar{t}}, h_t,t), t) \mathbb{E} \left[ \epsilon_t \dfrac{\partial V_{t+1}}{\partial K_{t+1}} \right]
\end{equation}

For each of our endogenous points, we have all of the elements except for $h$. Thus we can take the difference and set up a function we can minimize to solve for optimal labor supply.

In [11]:
# Define a function from eqn. 13 to use as the root finder.
def hFunc(h, Ktilde, uprime, age, E_eps_dvdk, b, eta, beta, d, alpha, omega2, omega0, omega1, delta):
    # print(h)
    tau = taufunc(age, omega0, omega1)
    kstart = psifunc(Ktilde, h, tau, d, alpha, omega2, delta)
    kappa = kappafunc(kstart, h, tau, d, omega2, alpha)
    dis = getMargDisutilityH(h, b, eta)
    ret = dis - uprime * kstart - beta * kappa * E_eps_dvdk
    # ret should be zero; we want to minimize, so take absolute value, and then multiply
    # since this number may be quite small
    ret = abs(ret) * 1e8
    return ret

# Now solve for optimal labor supply
for ixn in range(policy_df[:, :, ixt].shape[0]):
    if  policy_df[ixn, 6, ixt] == 1:
        policy_df[ixn, 7, ixt] = np.nan
        policy_df[ixn, 9, ixt] = np.nan 
    else:
        #print(ixn)
        Ktilde = policy_df[ixn, 1, ixt]
        this_E_eps_dVdK = policy_df[ixn, 4, ixt]
        thisuprime = policy_df[ixn, 8, ixt]
        out = scipy.optimize.minimize_scalar(hFunc, bounds=(1, hmax), method="bounded", args=(Ktilde, thisuprime, ixt+1+20,
                                                            this_E_eps_dVdK, b, eta, beta,
                                                            d, alpha, omega2, omega0, omega1, delta))
        policy_df[ixn, 9,  ixt] = out.x

In [12]:
# Now get the subset that is not liquidity constrained as a data frame
col_names = ["A_end", "K_end","E_uprime","E_dVdK","E_eps_dVdK","extrap","lc","C","uprime","H","K_start","A_start","dVdK"]
sub_policy_df = pd.DataFrame(policy_df[:, :, ixt], columns=col_names)
sub_policy_df = sub_policy_df[sub_policy_df['lc'] != 1]

In the last step, I drop all of the rows where the agent was liquidity constrained in period $T$. If this was the case, this means they were not following their FOCs, and thus the endogenous step we take is not valid. For purposes of filling in our exogenous grid, then, these observations are not helpful.

Here I need to comment on another shortcoming of my implementation of the EGM. If there are too few observations that are not liquidity constrained, either due to a poorly-set exogenous grid or to a strange parameterization of the problem, we cannot use the polynomial approximation to help fill in our exogenous grid. In my full implementation in R, all of the code I am presenting here is inside of a function called "SolveProblem()". If it is the case that during one of the endogenous steps there are fewer than 1/10 of the grid points that are done by interpolation, I stop the process and return a large number for the likelihood function. This is problematic inasmuch as it renders the likelihood function discontinuous, preventing me from using gradient-based optimization methods. However, as I currently implement the EGM, there may be some parameterizations where it is not possible to solve the agent's problem. My argument is that because I am setting a "reasonable" exogenous grid (matching what we find in the data), any parameterizaiton at which no solution can be reached cannot be a good one, and so my short-circuiting of the likelihood function only causes a problem for optimization, but will not prevent me from finding the true parameters.

With that out of the way, let's fill in the rest of the necessary information for our endogenous grid step. First, we can use our inverse laws of motion to get $\left\{A_{\underline{T-1}},K_{\underline{T-1}} \right\}$, and then will calculate $\mathbb{E} \epsilon \frac{\partial V}{\partial K}$.

In [13]:
# Use the inverted law of motion for K to get start-of-period HC
thistau = taufunc(ixt+1+20, omega0, omega1)
for i in range(sub_policy_df.shape[0]):
    # print(i)
    sub_policy_df.iloc[i, 10] = psifunc(sub_policy_df.iloc[i, 1], sub_policy_df.iloc[i, 9],thistau,d,alpha,omega2,delta)

# Now use the inverted IBC to get start-of-period A
for i in range(sub_policy_df.shape[0]):
    # print(i)
    sub_policy_df.iloc[i, 11] = sub_policy_df.iloc[i, 0] - sub_policy_df.iloc[i, 9]*sub_policy_df.iloc[i, 10] + sub_policy_df.iloc[i, 7]

# Now get dVdK
for i in range(sub_policy_df.shape[0]):
    # print(i)
    thislambda = lambdafunc(sub_policy_df.iloc[i, 9], thistau, omega2, d, alpha, delta)
    sub_policy_df.iloc[i, 12] = sub_policy_df.iloc[i, 9]*sub_policy_df.iloc[i, 8] + beta*thislambda*sub_policy_df.iloc[i, 4]

# policy_df index:
# 0 = end-of-period assets
# 1 = end-of-period capital
# 2 = E_uPrime
# 3 = E_dVdK
# 4 = E_eps_dVdK
# 5 = extrap
# 6 = lc
# 7 = C
# 8 = u'(C)
# 9 = H
# 10 = start-of-period capital
# 11 = start-of-period assets
# 12 = E_eps_dVdK

Our endogenous grid step has left us with a cross-section of data for the policy function. We now want to construct an exogenous, rectangular consumption grid from this data. The method currently used is essentially a Delauney triangulation.

In [14]:
# Now create a grid from our endogenous data points
y = a_grid[:,ixt]
x = k_grid[:,ixt]
xx, yy = np.meshgrid(x, y)
cgrid = griddata((sub_policy_df['K_start'], sub_policy_df['A_start']), sub_policy_df['C'], (xx, yy), method='linear')
policyC[:,:,ixt] = cgrid

Now let's create the polynomial approximation we'll use to fill in the values of our grid that are not covered by the endogenous grid points and are not liquidity constrained.

In [15]:
# create a data frame xx
xx = pd.DataFrame({'y1': sub_policy_df["C"],
                   'x1': sub_policy_df["A_start"],
                   'x2': sub_policy_df["K_start"]})
# Set up the polynomials of A and K
xx['x3'] = xx['x1'] * xx['x2']
xx['x4'] = (xx['x1'] ** 2) / 1e5  # assets
xx['x5'] = xx['x2'] ** 2

# get the model
out_C2 = sm.OLS(xx['y1'], xx[['x1', 'x2', 'x3', 'x4', 'x5']]).fit()

# get the relevant portion of the polynomial df
# set up the data frame with all the polynomials ahead of time
col_names = ["x1","x2","x3","x4","x5"]
polynomialdf = pd.DataFrame(polynomial_df[:,:,ixt], columns=col_names)
# polynomialdf

Now we will fill in the rest of our exogenous grid: those points where the agent is liquidity constrained, and those points which the endogenous grid did not cover. In order to get optimal H from optimal C, we will define a function using the FOC for labor supply which we can plug into a root finder.

In [16]:
# First, cast our slices for interpolation as contiguous arrays
K_slice = np.ascontiguousarray(k_grid[:,ixt+1])
A_slice = np.ascontiguousarray(a_grid[:,ixt+1])
E_eps_dVdK_slice = np.ascontiguousarray(E_eps_dVdK[:,:,ixt+1])

# and define our interpolator function
interp_func_eps_dVdK = RegularGridInterpolator((K_slice, A_slice), E_eps_dVdK_slice, method='linear')

def hFuncZero(K,c,h,age,E_eps_dvdk,b,eta,gamma,beta,d, omega2,alpha,omega0,omega1):
    thistau = taufunc(age, omega0, omega1)
    kappa = kappafunc(K, h, thistau, d, omega2, alpha)
    dis = getMargDisutilityH(h, b, eta)
    util = getMargUtilityC(c,gamma)
    ret = dis - util * K - beta * kappa * E_eps_dvdk
    return ret

def FOCzero(h,c,age,A,K,gamma,b,eta,omega0,omega1,d,alpha,omega2,delta,beta,E_eps_dVdK,sub_agrid,sub_kgrid):
    hclb =np.min(sub_kgrid)
    hcub =np.max(sub_kgrid)
    alb =np.min(sub_agrid)
    aub =np.max(sub_agrid)

    # next period assets given this choice
    Aend = R*(A+h*K - c)
    # Check it doesn't violate our bounds so that we can interpolate
    # Aend = np.where(Aend > alb, Aend, alb+0.1)
    # Aend = np.where(Aend < aub, Aend, aub-0.1)

    # deterministic part of human capital:
    Kend = gFunc(h,K,age,delta,omega0,omega1,alpha,omega2,d).item()
    # Check it doesn't violate our bounds so that we can interpolate
    Kend = np.where(Kend > hclb, Kend, hclb+0.1).item()
    Kend = np.where(Kend < hcub, Kend, hcub-0.1).item()

    if (Aend < aub) & (Aend > alb):
        # get the value of E_eps_dVdK for next period
        # this_epsdVdKnext = interp2d(sub_kgrid, sub_agrid, E_eps_dVdK, kind='linear')(Kend, Aend)
        thispoint = (Kend, Aend)
        this_epsdVdKnext = interp_func_eps_dVdK(thispoint)

        # solve problem
        val = hFuncZero(K, c, h, age, this_epsdVdKnext, b, eta, gamma, beta, d, omega2, alpha, omega0, omega1)
    else:
        val = np.where(Aend <= alb, -(Aend*1.2)*1e2, (Aend*1.2)*1e2)
    
    return val*10

# Now let's go through our grid points
for ixa in range(nA): # looping through asset points
    for ixk in range(nK): # looping through human capital points
        if ixt == 43:
            cub = hmax*k_grid[ixk,ixt] + a_grid[ixa,ixt] + (2040*k_grid[ixk,ixt])*(1/R)
        elif ixt < 43:
            cub = hmax*k_grid[ixk,ixt] + a_grid[ixa,ixt] + (2*2040*k_grid[ixk,ixt])*(1/R)

        if cub < cmin:
            # liquidity constrained
            policyC[ixa,ixk,ixt] = cmin
            policyH[ixa,ixk,ixt] = hmax
            lc[ixa,ixk,ixt] = 1
        
            # fill in uPrime and dVdK
            E_eps_dVdK_tp1 = np.max(E_eps_dVdK[:,:,ixt+1])
            uPrime[ixa,ixk,ixt] = uprimemin
            dVdK[ixa,ixk,ixt]   = hmax*uprimemin + beta*lambdafunc(hmax,taufunc(ixt+1+20,omega0,omega1),omega2,d,alpha,delta)*E_eps_dVdK_tp1
        else:
            if np.isnan(policyC[ixa,ixk,ixt]) or (ixa > 1 and (not np.isnan(policyC[ixa,ixk,ixt]) and policyC[ixa-1,ixk,ixt] > policyC[ixa,ixk,ixt])):
                extrap[ixa,ixk,ixt] = 1
                nxx = polynomialdf[(polynomialdf['x1'] == a_grid[ixa,ixt]) & (polynomialdf['x2'] == k_grid[ixk,ixt])]
                policyC[ixa,ixk,ixt] = out_C2.predict(nxx) 

            # make sure c is not over our upper bound or below our lower bound
            c = np.where(policyC[ixa,ixk,ixt] > cmin, policyC[ixa,ixk,ixt], cmin+.1).item()
            c = np.where(c < cub, c, cub-.1).item()
            policyC[ixa,ixk,ixt] = c
            
            # catch occasions where the polynomial predicts lower consumption with assets
            if ixa > 0:
                if policyC[ixa-1,ixk,ixt] > policyC[ixa,ixk,ixt]:
                    c = policyC[ixa-1,ixk,ixt]*1.01
                    policyC[ixa,ixk,ixt] = c
                    replace[ixa,ixk,ixt] = 1

            # Now we have optimal C and move on to optimal H
            # get minimum hours
            # now use our consumption choice to get minimum hours
            if ixt == 43:
                hmin = (-(2040 * k_grid[ixk, ixt]) * (1 / R) - a_grid[ixa, ixt] + c) / k_grid[ixk, ixt] + 0.1
            elif ixt < 43:
                hmin = (-(2 * 2040 * k_grid[ixk, ixt]) * (1 / R) - a_grid[ixa, ixt] + c) / k_grid[ixk, ixt] + 0.1

            # Make sure hmin is positive but less than hmax
            hmin = np.where(hmin > 0, hmin, 0.1).item()
            hmin = np.where(hmin >= hmax, hmax-.1, hmin).item()
            
            p1 = FOCzero(hmin,c,ixt+1+20,a_grid[ixa,ixt],k_grid[ixk,ixt],gamma,b,eta,omega0,omega1,d,alpha,omega2,delta,beta,
                         E_eps_dVdK_slice, A_slice, K_slice).item()
            p2 = FOCzero(hmax,c,ixt+1+20,a_grid[ixa,ixt],k_grid[ixk,ixt],gamma,b,eta,omega0,omega1,d,alpha,omega2,delta,beta,
                         E_eps_dVdK_slice, A_slice, K_slice).item()
            if p1*p2>0: # either both positive or both negative
              h = hmin if abs(p1)<abs(p2) else hmax 
            elif p1*p2 < 0: # endpoints are of different signs
              out = root_scalar(FOCzero, bracket=[hmin, hmax], args=(c,ixt+1+20,a_grid[ixa,ixt],k_grid[ixk,ixt],
                                                                     gamma,b,eta,omega0,omega1,d,alpha,omega2,delta,beta,
                                                                     E_eps_dVdK_slice,A_slice, K_slice), 
                                                                     method='brentq', options={'maxiter': 500, 'xtol': 1e-12}) 
              h = out.root
            
            else: # case where p1*p2=0 (almost surely won't happen)
              h = hmin if p1==0 else hmax
            # now fill in the policy functions
            policyH[ixa,ixk,ixt] = h
            lc[ixa,ixk,ixt] = 0

            # now fill in Uprime and dVdK

            # next period HC
            thisK = k_grid[ixk,ixt]
            Knext = gFunc(h,thisK,ixt+1+20,delta,omega0,omega1,alpha,omega2,d)
            Knext = np.where(Knext > np.min(k_grid[:,ixt+1]), Knext, np.min(k_grid[:,ixt+1])+0.1).item()
            Knext = np.where(Knext < np.max(k_grid[:,ixt+1]), Knext, np.max(k_grid[:,ixt+1])-0.1).item()

            # next period A
            Anext = a_grid[ixa,ixt] + h*thisK - c
            Anext = np.where(Anext > np.min(a_grid[:,ixt+1]), Anext, np.min(a_grid[:,ixt+1])+0.1).item()
            Anext = np.where(Anext < np.max(a_grid[:,ixt+1]), Anext, np.max(a_grid[:,ixt+1])-0.1).item()
            
            # now interpolate E[eps_dVdK]_{t+1}
            point = ( Knext, Anext)
            E_eps_dVdK_tp1 = interp_func_eps_dVdK(point).item()
            uPrime[ixa,ixk,ixt] = getMargUtilityC(c,gamma)
            dVdK[ixa,ixk,ixt]   = h*uPrime[ixa,ixk,ixt] + beta*lambdafunc(h,taufunc(ixt+1+20,omega0,omega1), omega2, d, alpha,delta)*E_eps_dVdK_tp1
    # Once we have looped through human capital once and filled in the grid for a given level of assets, we can loop through again and fill in our expectations grid.
    
# set up our interpolator functions for the expectations grid
    interp_func_uprime = interpolate.interp1d(k_grid[:,ixt], uPrime[ixa,:,ixt])
    interp_func_dVdK = interpolate.interp1d(k_grid[:,ixt], dVdK[ixa,:,ixt])

    for ixk in range(nK):
         nextK = quad*k_grid[ixk,ixt]
         # If HC is off the grid, force it to stay on.
         nextK = np.where(nextK <= k_grid[0,ixt], k_grid[0,ixt]+0.01, nextK)
         nextK = np.where(nextK >= k_grid[nK-1,ixt], k_grid[nK-1,ixt]-0.01, nextK)

         Uprime_nextvec = interp_func_uprime(nextK)
         dVdK_nextvec  = interp_func_dVdK(nextK)
         for i in range(len(dVdK_nextvec)):
              eps_dVdK_nextvec[i] = quad[i] * dVdK_nextvec[i]
      
         E_uPrime[ixa,ixk,ixt] = np.mean(Uprime_nextvec)
         E_dVdK[ixa,ixk,ixt] = np.mean(dVdK_nextvec)
         E_eps_dVdK[ixa,ixk,ixt] = np.mean(eps_dVdK_nextvec)

We are now done solving the problem in period $T-1$. We have an approximation of our policy function on an exogenous grid over which we can interpolate when we want to simulate our data, and we have all of the other objects we will need to perform the endogenous grid step in period $T-2$, and iterate this process backwards to the initial period. In the next cell, I will loop over all the previous time periods and do just this.

In [17]:
for ixt in range(lifespan-3, -1, -1):
    # Melt our previous values from nexy period to use in this one
    # end of period assets
    policy_df[:,0,ixt] = np.tile(a_grid[:,ixt+1]*(R**(-1)),len(k_grid[:,ixt+1])) # unwind the interest payment to get assets at end of T-1

    # end of period human capital
    policy_df[:,1,ixt] = np.kron(k_grid[:,ixt+1], np.repeat(1,len(a_grid[:,ixt+1])))
    # here we are treating realized HC in T as the deterministic HC at the end of T-1

    # now melt the expected values we calculated
    df = pd.DataFrame(E_uPrime[:, :, ixt+1])
    policy_df[:,2, ixt] = pd.melt(df)["value"]
    df = pd.DataFrame(E_dVdK[:, :, ixt+1])
    policy_df[:,3, ixt] = pd.melt(df)["value"]
    df = pd.DataFrame(E_eps_dVdK[:, :, ixt+1])
    policy_df[:,4, ixt] = pd.melt(df)["value"]

    # a few other values
    df = pd.DataFrame(extrap[:, :, ixt+1])
    policy_df[:,5, ixt] = pd.melt(df)["value"]
    df = pd.DataFrame(lc[:, :, ixt+1])
    policy_df[:,6, ixt] = pd.melt(df)["value"]

    # Now get that period's optimal consumption using the inverted FOC
    policy_df[:, 7, ixt] = (beta*R*policy_df[:,2,ixt])**(1/(-gamma));
    # and get marginal utility:
    for i in range(policy_df.shape[0]):
        policy_df[i,8, ixt] = getMargUtilityC(policy_df[i, 7, ixt], gamma)

    # Now solve for optimal labor supply
    for ixn in range(policy_df[:, :, ixt].shape[0]):
        if  policy_df[ixn, 6, ixt] == 1:
            policy_df[ixn, 7, ixt] = np.nan
            policy_df[ixn, 9, ixt] = np.nan 
        else:
            Ktilde = policy_df[ixn, 1, ixt]
            this_E_eps_dVdK = policy_df[ixn, 4, ixt]
            thisuprime = policy_df[ixn, 8, ixt]
            out = scipy.optimize.minimize_scalar(hFunc, bounds=(1, hmax), method="bounded", args=(Ktilde, thisuprime, ixt+1+20,
                                                                this_E_eps_dVdK, b, eta, beta,
                                                                d, alpha, omega2, omega0, omega1, delta))
            policy_df[ixn, 9,  ixt] = out.x

    # Turn into a data frame
    col_names = ["A_end", "K_end","E_uprime","E_dVdK","E_eps_dVdK","extrap","lc","C","uprime","H","K_start","A_start","dVdK"]
    # sub_policy_df = pd.DataFrame(policy_df[policy_df[:, 6, lifespan-1-1] != 1, :, lifespan-1-1], columns=col_names)
    sub_policy_df = pd.DataFrame(policy_df[:, :, ixt], columns=col_names)
    sub_policy_df = sub_policy_df[sub_policy_df['lc'] != 1]

    # Work backwards to get start-of-period K
    thistau = taufunc(ixt+1+20, omega0, omega1)
    for i in range(sub_policy_df.shape[0]):
        # print(i)
        sub_policy_df.iloc[i, 10] = psifunc(sub_policy_df.iloc[i, 1], sub_policy_df.iloc[i, 9],thistau,d,alpha,omega2,delta)

    # Now use the inverted IBC to get start-of-period A
    for i in range(sub_policy_df.shape[0]):
        sub_policy_df.iloc[i, 11] = sub_policy_df.iloc[i, 0] - sub_policy_df.iloc[i, 9]*sub_policy_df.iloc[i, 10] + sub_policy_df.iloc[i, 7]

    # Now get dVdK
    for i in range(sub_policy_df.shape[0]):
        thislambda = lambdafunc(sub_policy_df.iloc[i, 9], thistau, omega2, d, alpha, delta)
        sub_policy_df.iloc[i, 12] = sub_policy_df.iloc[i, 9]*sub_policy_df.iloc[i, 8] + beta*thislambda*sub_policy_df.iloc[i, 4]

    # Now create a grid from our endogenous data points
    y = a_grid[:,ixt]
    x = k_grid[:,ixt]
    xx, yy = np.meshgrid(x, y)

    cgrid = griddata((sub_policy_df['K_start'], sub_policy_df['A_start']), sub_policy_df['C'], (xx, yy), method='linear')
    policyC[:,:,ixt] = cgrid

    # Now setup the polynomial approximation for the policy function
    xx = pd.DataFrame({'y1': sub_policy_df["C"],
                    'x1': sub_policy_df["A_start"],
                    'x2': sub_policy_df["K_start"]})
    # Set up the polynomials of A and K
    xx['x3'] = xx['x1'] * xx['x2']
    xx['x4'] = (xx['x1'] ** 2) / 1e5  # assets
    xx['x5'] = xx['x2'] ** 2

    # get the model
    out_C2 = sm.OLS(xx['y1'], xx[['x1', 'x2', 'x3', 'x4', 'x5']]).fit()

    # get the relevant portion of the polynomial df
    # set up the data frame with all the polynomials ahead of time
    col_names = ["x1","x2","x3","x4","x5"]
    polynomialdf = pd.DataFrame(polynomial_df[:,:,ixt], columns=col_names)

    # Now go through all of our grid points
    # First, cast our slices for interpolation as contiguous arrays
    K_slice = np.ascontiguousarray(k_grid[:,ixt+1])
    A_slice = np.ascontiguousarray(a_grid[:,ixt+1])
    E_eps_dVdK_slice = np.ascontiguousarray(E_eps_dVdK[:,:,ixt+1])
    # and define our interpolator function
    interp_func_eps_dVdK = RegularGridInterpolator((K_slice, A_slice), E_eps_dVdK_slice, method='linear')

    for ixa in range(nA): # looping through asset points
        for ixk in range(nK): # looping through human capital points
            if ixt == 43:
                cub = hmax*k_grid[ixk,ixt] + a_grid[ixa,ixt] + (2040*k_grid[ixk,ixt])*(1/R)
            elif ixt < 43:
                cub = hmax*k_grid[ixk,ixt] + a_grid[ixa,ixt] + (2*2040*k_grid[ixk,ixt])*(1/R)

            if cub < cmin:
                # liquidity constrained
                policyC[ixa,ixk,ixt] = cmin
                policyH[ixa,ixk,ixt] = hmax
                lc[ixa,ixk,ixt] = 1
            
                # fill in uPrime and dVdK
                E_eps_dVdK_tp1 = np.max(E_eps_dVdK[:,:,ixt+1])
                uPrime[ixa,ixk,ixt] = uprimemin
                dVdK[ixa,ixk,ixt]   = hmax*uprimemin + beta*lambdafunc(hmax,taufunc(ixt+1+20,omega0,omega1),omega2,d,alpha,delta)*E_eps_dVdK_tp1
            else:
                if np.isnan(policyC[ixa,ixk,ixt]) or (ixa > 1 and (not np.isnan(policyC[ixa,ixk,ixt]) and policyC[ixa-1,ixk,ixt] > policyC[ixa,ixk,ixt])):
                    extrap[ixa,ixk,ixt] = 1
                    nxx = polynomialdf[(polynomialdf['x1'] == a_grid[ixa,ixt]) & (polynomialdf['x2'] == k_grid[ixk,ixt])]
                    policyC[ixa,ixk,ixt] = out_C2.predict(nxx) 

                # make sure c is not over our upper bound or below our lower bound
                c = np.where(policyC[ixa,ixk,ixt] > cmin, policyC[ixa,ixk,ixt], cmin+.1).item()
                c = np.where(c < cub, c, cub-.1).item()
                policyC[ixa,ixk,ixt] = c
                
                # catch occasions where the polynomial predicts lower consumption with assets
                if ixa > 0:
                    if policyC[ixa-1,ixk,ixt] > policyC[ixa,ixk,ixt]:
                        c = policyC[ixa-1,ixk,ixt]*1.01
                        policyC[ixa,ixk,ixt] = c
                        replace[ixa,ixk,ixt] = 1

                # Now we have optimal C and move on to optimal H
                # get minimum hours
                # now use our consumption choice to get minimum hours
                if ixt == 43:
                    hmin = (-(2040 * k_grid[ixk, ixt]) * (1 / R) - a_grid[ixa, ixt] + c) / k_grid[ixk, ixt] + 0.1
                elif ixt < 43:
                    hmin = (-(2 * 2040 * k_grid[ixk, ixt]) * (1 / R) - a_grid[ixa, ixt] + c) / k_grid[ixk, ixt] + 0.1

                # Make sure hmin is positive but less than hmax
                hmin = np.where(hmin > 0, hmin, 0.1).item()
                hmin = np.where(hmin >= hmax, hmax-.1, hmin).item()
                
                p1 = FOCzero(hmin,c,ixt+1+20,a_grid[ixa,ixt],k_grid[ixk,ixt],gamma,b,eta,omega0,omega1,d,alpha,omega2,delta,beta,
                             E_eps_dVdK_slice, A_slice, K_slice).item()
                p2 = FOCzero(hmax,c,ixt+1+20,a_grid[ixa,ixt],k_grid[ixk,ixt],gamma,b,eta,omega0,omega1,d,alpha,omega2,delta,beta,
                             E_eps_dVdK_slice, A_slice, K_slice).item()
                if p1*p2>0: # either both positive or both negative
                    h = hmin if abs(p1)<abs(p2) else hmax 
                elif p1*p2 < 0: # endpoints are of different signs
                    out = root_scalar(FOCzero, bracket=[hmin, hmax], args=(c,ixt+1+20,a_grid[ixa,ixt],k_grid[ixk,ixt],
                                                                        gamma,b,eta,omega0,omega1,d,alpha,omega2,delta,beta,
                                                                        E_eps_dVdK_slice,A_slice, K_slice),
                                                                        method='brentq', options={'maxiter': 500, 'xtol': 1e-12})
                    h = out.root
                else: # case where p1*p2=0 (almost surely won't happen)
                    h = hmin if p1==0 else hmax
                # now fill in the policy functions
                policyH[ixa,ixk,ixt] = h
                lc[ixa,ixk,ixt] = 0

                # now fill in Uprime and dVdK

                # next period HC
                thisK = k_grid[ixk,ixt]
                Knext = gFunc(h,thisK,ixt+1+20,delta,omega0,omega1,alpha,omega2,d)
                Knext = np.where(Knext > np.min(k_grid[:,ixt+1]), Knext, np.min(k_grid[:,ixt+1])+0.1).item()
                Knext = np.where(Knext < np.max(k_grid[:,ixt+1]), Knext, np.max(k_grid[:,ixt+1])-0.1).item()

                # next period A
                Anext = a_grid[ixa,ixt] + h*thisK - c
                Anext = np.where(Anext > np.min(a_grid[:,ixt+1]), Anext, np.min(a_grid[:,ixt+1])+0.1).item()
                Anext = np.where(Anext < np.max(a_grid[:,ixt+1]), Anext, np.max(a_grid[:,ixt+1])-0.1).item()
                
                # now interpolate E[eps_dVdK]_{t+1}
                point = ( Knext, Anext) # (Knext,Anext)
                E_eps_dVdK_tp1 = interp_func_eps_dVdK(point).item()
                uPrime[ixa,ixk,ixt] = getMargUtilityC(c,gamma)
                dVdK[ixa,ixk,ixt]   = h*uPrime[ixa,ixk,ixt] + beta*lambdafunc(h,taufunc(ixt+1+20,omega0,omega1), omega2, d, alpha,delta)*E_eps_dVdK_tp1
        # Once we have looped through human capital once and filled in the grid for a given level of assets, we can loop through again and fill in our expectations grid.
    
    # set up our interpolator functions for the expectations grid
        interp_func_uprime = interpolate.interp1d(k_grid[:,ixt], uPrime[ixa,:,ixt])
        interp_func_dVdK = interpolate.interp1d(k_grid[:,ixt], dVdK[ixa,:,ixt])

        for ixk in range(nK):
            nextK = quad*k_grid[ixk,ixt]
            # If HC is off the grid, force it to stay on.
            nextK = np.where(nextK <= k_grid[0,ixt], k_grid[0,ixt]+0.01, nextK)
            nextK = np.where(nextK >= k_grid[nK-1,ixt], k_grid[nK-1,ixt]-0.01, nextK)

            Uprime_nextvec = interp_func_uprime(nextK)
            dVdK_nextvec  = interp_func_dVdK(nextK)
            for i in range(len(dVdK_nextvec)):
                eps_dVdK_nextvec[i] = quad[i] * dVdK_nextvec[i]
        
            E_uPrime[ixa,ixk,ixt] = np.mean(Uprime_nextvec)
            E_dVdK[ixa,ixk,ixt] = np.mean(dVdK_nextvec)
            E_eps_dVdK[ixa,ixk,ixt] = np.mean(eps_dVdK_nextvec)

42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
